In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Read the CSV file
df = pd.read_csv('R1.csv')
df.head(100)

,Test Cases Name,Test Cases,Requirements,Dependency,Fault Severity,Release1/Fault Count,Verdict
0,test case addition failed in tester's log test...,T1,R10,3,0,0,1
1,add bugs in tester's log test suite,T2,R10,3,0,0,1
2,add personal projects in tester's portfolio,T3,R9,3,0,0,1
3,add requirement button on client project initi...,T4,R6,3,0,0,1
4,add skills in tester's portfolio,T5,R9,3,1,1,0
...,...,...,...,...,...,...,...
86,tester's dashboard progress button,T87,R14,3,1,1,0
87,tester's profile updation,T88,R13,3,0,0,1
88,update uploaded resume,T89,R13,3,0,0,1
89,upload resume button in tester's portfolio,T90,R9,3,2,1,0


In [3]:
# Get the shape of the DataFrame
shape = df.shape
# Print the number of rows and columns
print("Number of rows:", shape[0])
print("Number of columns:", shape[1])

df.columns

Number of rows: 91
Number of columns: 7


Index(['Test Cases Name', 'Test Cases ', 'Requirements', 'Dependency',
       'Fault Severity', 'Release1/Fault Count', 'Verdict'],
      dtype='object')

In [4]:
# Extract the test cases from the CSV
test_cases = df['Test Cases ']

# Extract the feature columns from the CSV
feature_columns = df.columns[3:]

# Create a dictionary to store the features and their values
features = {}
for feature in feature_columns:
    features[feature] = df[feature].values

In [5]:
# Define the environment
class TestPrioritizationEnv:
    def __init__(self, test_cases, features):
        self.test_cases = test_cases
        self.features = features
        self.state_size = len(features) 
        self.action_size = len(test_cases)
        self.current_test = 0

    def reset(self):
        self.current_test = 0
        return self.get_state()
    
    def get_state(self):
        if self.current_test < len(self.test_cases):
            state = [self.features[feature][self.current_test] for feature in self.features]
            return state
        else:
            
            return None

    def step(self, action):
        test_case = self.test_cases[self.current_test]
        reward = self.calculate_business_value(test_case)
        self.current_test += 1
        done = self.current_test == len(self.test_cases)
        return self.get_state(), reward, done

    def calculate_business_value(self, test_case):
        index = self.test_cases.tolist().index(test_case)
        business_value = sum([self.features[feature][index] for feature in self.features])
        return business_value
    

In [6]:
class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.learning_rate = learning_rate
        self.model = self.build_model()
    
    def build_model(self):
        model = Sequential()
        model.add(Dense(self.state_size, input_dim=self.state_size, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

  
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))  

    
    def act(self, state):
        state = np.array(state).reshape(1, -1)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
 
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = np.array(state)  
            state = np.reshape(state, [1, self.state_size])

            if next_state is not None:
                next_state = np.array(next_state)  
                next_state = np.reshape(next_state, [1, self.state_size])
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            else:
               
                target = reward  

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)


    def train(self, env, episodes, batch_size):
        for episode in range(episodes):
            state = env.reset()
            total_reward = 0
            done = False
            while not done:
                action = self.act(state)
                next_state, reward, done = env.step(action)
                self.remember(state, action, reward, next_state, done)
                total_reward += reward
                state = next_state

            if len(self.memory) > batch_size:
                self.replay(batch_size)

    def get_prioritized_test_cases(self, env):
        prioritized_test_cases = []
        state = env.reset()
        done = False
        while not done:
            action = self.act(state)
            test_case = env.test_cases[env.current_test]
            reward = env.calculate_business_value(test_case)
            prioritized_test_cases.append((test_case, reward))
            next_state, _, done = env.step(action)
            state = next_state
        return prioritized_test_cases

In [7]:
# Create the environment with the test cases and features
env = TestPrioritizationEnv(test_cases, features)

# Create the DQN agent
state_size = env.state_size
action_size = env.action_size
agent = DQNAgent(state_size, action_size)

# Train the DQN agent
episodes = 10
batch_size = 32
agent.train(env, episodes, batch_size)


1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 39ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 59ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 31ms/step


In [8]:
# Calculate the business value and prioritize the test cases
prioritized_test_cases = []
for test_case in test_cases:
    business_value = env.calculate_business_value(test_case) # calculation of reward as business value
    prioritized_test_cases.append((test_case, business_value))

# Sort the test cases based on business value (in descending order)
prioritized_test_cases.sort(key=lambda x: x[1], reverse=True)

# Print the prioritized test cases
for test_case, business_value in prioritized_test_cases:
    print(f"Test Case: {test_case}")

Test Case: T29
Test Case: T35
Test Case: T39
Test Case: T21
Test Case: T25
Test Case: T27
Test Case: T43
Test Case: T81
Test Case: T90
Test Case: T5
Test Case: T10
Test Case: T61
Test Case: T62
Test Case: T64
Test Case: T65
Test Case: T87
Test Case: T1
Test Case: T2
Test Case: T3
Test Case: T4
Test Case: T6
Test Case: T7
Test Case: T8
Test Case: T9
Test Case: T11
Test Case: T13
Test Case: T14
Test Case: T22
Test Case: T23
Test Case: T24
Test Case: T26
Test Case: T28
Test Case: T31
Test Case: T33
Test Case: T34
Test Case: T37
Test Case: T44
Test Case: T49
Test Case: T50
Test Case: T51
Test Case: T52
Test Case: T53
Test Case: T54
Test Case: T55
Test Case: T56
Test Case: T57
Test Case: T58
Test Case: T59
Test Case: T60
Test Case: T66
Test Case: T67
Test Case: T68
Test Case: T69
Test Case: T70
Test Case: T71
Test Case: T72
Test Case: T73
Test Case: T74
Test Case: T75
Test Case: T76
Test Case: T77
Test Case: T78
Test Case: T79
Test Case: T80
Test Case: T85
Test Case: T88
Test Case: T89
Test